In [1]:
# Run our imports
import pandas as pd
import numpy as np
import yfinance as yf
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
import re


# INPUTS

In [2]:
# select Ticker
tickers = ['TSLA','AAPL','MSFT']
ticker = 'TSLA'
# Target web page:
url = f'https://seekingalpha.com/symbol/{ticker}/news'
# url = 'file:///Users/danielgurzi/Desktop/Tesla,%20Inc.%20(TSLA)%20Latest%20Stock%20News%20_%20Seeking%20Alpha.html'

# DATA GATHERING

In [3]:
# take ticker list, output csvs with stock price info
def tick_hist(tickers):
    for t in tickers:
        df = pd.DataFrame(yf.Ticker(t).history())
        df.to_csv(f'./data/{t}_history.csv', index=False)
    return

In [ ]:
tick_hist(tickers)

Need to make a function for cathering article hrefs

Needs to update the date of Today and Yesterday to proper dates



In [4]:
# this code is used to get the article lists from the www.seekingalpha.com/{ticker}/news
driver = webdriver.Chrome('./chromedriver')
driver.get(url)
driver.implicitly_wait(100)



In [6]:
# set the driver to grab the links
links = driver.find_elements_by_class_name("_1-r1S")

In [7]:
# This dataframe will organize the links to the articles from the ticker
df = pd.DataFrame()
df['date'] = [i.find_element_by_class_name("VbEwc").text for i in links]
df['name'] = [i.find_element_by_tag_name("h3").text for i in links]
df['href'] = [i.find_elements_by_tag_name('a')[0].get_property('href') for i in links]
# df['comments_qty'] = [i.find_element_by_class_name('_3zL6Q').text if i.find_element_by_class_name('_3zL6Q').text else '0 comments' for i in links]

In [8]:
# this will end the browser session
driver.close()


In [9]:
df

,date,name,href
0,"Today, 3:48 PM",Sunrun deal for Vivint to top Tesla as rooftop...,https://seekingalpha.com/news/3589237-sunrun-d...
1,"Today, 10:21 AM",Tesla bears capitulate for the near term,https://seekingalpha.com/news/3589120-tesla-be...
2,"Yesterday, 1:16 PM",Tesla market cap crown brings focus on valuation,https://seekingalpha.com/news/3588818-tesla-ma...
3,"Yesterday, 8:41 AM","Tesla +6% after JMP hikes price target to $1,500",https://seekingalpha.com/news/3588673-teslaplu...
4,"Yesterday, 3:26 AM",Musk teases Tesla shortsellers with 'Short Sho...,https://seekingalpha.com/news/3588579-musk-tea...
...,...,...,...
315,"Mon, Nov. 25, 2019",Cybertruck orders trigger Tesla rebound,https://seekingalpha.com/news/3521690-cybertru...
316,"Sun, Nov. 24, 2019",146K orders placed for Tesla Cybertruck,https://seekingalpha.com/news/3521667-146k-ord...
317,"Fri, Nov. 22, 2019",Funko and Modine Manufacturing among Consumer ...,https://seekingalpha.com/news/3521591-funko-an...
318,"Fri, Nov. 22, 2019",Tesla -4% after Cybertruck debut,https://seekingalpha.com/news/3521515-teslamin...


In [ ]:
# save info to dataframe
df.to_csv(f'./data/{ticker}_link_history.csv')


In [10]:
# Clean it up to take out special characters from the article name
# df['name'] = [i.replace('/', ' ')]
df['name'] = [re.sub('[^A-Za-z0-9 $]+', '', i) for i in df['name']]
df.head()

,date,name,href
0,"Today, 3:48 PM",Sunrun deal for Vivint to top Tesla as rooftop...,https://seekingalpha.com/news/3589237-sunrun-d...
1,"Today, 10:21 AM",Tesla bears capitulate for the near term,https://seekingalpha.com/news/3589120-tesla-be...
2,"Yesterday, 1:16 PM",Tesla market cap crown brings focus on valuation,https://seekingalpha.com/news/3588818-tesla-ma...
3,"Yesterday, 8:41 AM",Tesla 6 after JMP hikes price target to $1500,https://seekingalpha.com/news/3588673-teslaplu...
4,"Yesterday, 3:26 AM",Musk teases Tesla shortsellers with Short Shorts,https://seekingalpha.com/news/3588579-musk-tea...


### Now we have a list of URLS from the seeking alpha ticker page

In [ ]:
for r,l in df.iterrows():
    print(l[1])

In [ ]:
# open the selenium driver to get the actual article link and save the page information as a file.
driver = webdriver.Chrome('./chromedriver')
for r,c in df.iterrows():
    driver.get(c[2])
    with open(f"./data/{ticker}_history/{c[1]}", "w") as f:
        f.write(driver.page_source)
driver.close()



In [11]:
# this function will take the article links dataframe and go get the articles themselves 
# and save the information to a new dataframe

def qurom(df):
    #start a list of the articles
    art_list = []
    # grap the comments while I'm here
    comments = []
    # for loop to go through each of the links and grab the articles
    for l in df['href']:
        driver.get(l)
        driver.implicitly_wait(1)
        try: 
            article = driver.find_element_by_id("a-cont")
        except:
            article = 'error'
        # The Article has comments, which for now I am going to take out. This code will separate the article into two sections with "Like this article" as the separator
        sep = 'Recommended for you:'
        try: 
            art_list.append(article.text.split(sep,1)[0])
        except:
            art_list.append('error')
        try: 
            comments.append(article.text.split(sep,1)[1])
        except:
            comments.append('error')
    df['article'] = art_list
    df['comments'] = comments
    return df

In [12]:
# Open the Selenium Driver and run the qurom function. 
driver = webdriver.Chrome('./chromedriver')

df2 = qurom(df)
# df.to_csv(f'./data/{ticker}.csv', index = False)

# close the browser


In [19]:
# Close the Selenium Browser
driver.close()


InvalidSessionIdException: Message: invalid session id


In [14]:
df

,date,name,href,article,comments
0,"Today, 3:48 PM",Sunrun deal for Vivint to top Tesla as rooftop...,https://seekingalpha.com/news/3589237-sunrun-d...,Sunrun (RUN +23.9%) surges to all-time highs a...,error
1,"Today, 10:21 AM",Tesla bears capitulate for the near term,https://seekingalpha.com/news/3589120-tesla-be...,The bear camp on Tesla (TSLA +0.4%) is crackin...,error
2,"Yesterday, 1:16 PM",Tesla market cap crown brings focus on valuation,https://seekingalpha.com/news/3588818-tesla-ma...,Tesla shares got a lot of hype last week as th...,error
3,"Yesterday, 8:41 AM",Tesla 6 after JMP hikes price target to $1500,https://seekingalpha.com/news/3588673-teslaplu...,JMP Securities thinks sales volume at Tesla (N...,error
4,"Yesterday, 3:26 AM",Musk teases Tesla shortsellers with Short Shorts,https://seekingalpha.com/news/3588579-musk-tea...,error,error
...,...,...,...,...,...
315,"Mon, Nov. 25, 2019",Cybertruck orders trigger Tesla rebound,https://seekingalpha.com/news/3521690-cybertru...,error,error
316,"Sun, Nov. 24, 2019",146K orders placed for Tesla Cybertruck,https://seekingalpha.com/news/3521667-146k-ord...,error,error
317,"Fri, Nov. 22, 2019",Funko and Modine Manufacturing among Consumer ...,https://seekingalpha.com/news/3521591-funko-an...,error,error
318,"Fri, Nov. 22, 2019",Tesla 4 after Cybertruck debut,https://seekingalpha.com/news/3521515-teslamin...,error,error


In [15]:
def finish_qurom(df):
    driver = webdriver.Chrome('./chromedriver')
    #start a list of the articles
    art_list = []
    # grap the comments while I'm here
    comments = []
    # for loop to go through each of the links and grab the articles
    for r,c in df.iterrows():
        if c[3] == 'error':
            sep = 'Recommended for you:'
            driver.get(c[2])
#             driver.implicitly_wait(30)
            try:
                driver.find_element_by_class_name('premium-banner')
            except:
                df['article'][r] = 'no_good'
            try: 
                df['article'][r] = driver.find_element_by_id("a-cont").text.split(sep,1)[0]
            except:
                article = 'error'
            try: 
                df['comments'][r] = driver.find_element_by_id("a-cont").text.split(sep,1)[1]
            except:
                comments.append('error')
    driver.close()        
    
    return df

In [ ]:
df2[df2['name'].str.contains("volatile market")]
df2.drop(df2.index[67], inplace=True)

In [17]:
if 'error' in df2['article'].unique():
    print(df2['article'].str.count('error').sum())
    finish_qurom(df2)
else:
    print('Done')


Done


In [20]:
# save new dataframe to a csv
df2.to_csv(f'./data/{ticker}_articles.csv', index = False)


In [18]:
df2.head()

,date,name,href,article,comments
0,"Today, 3:48 PM",Sunrun deal for Vivint to top Tesla as rooftop...,https://seekingalpha.com/news/3589237-sunrun-d...,Sunrun (RUN +23.9%) surges to all-time highs a...,error
1,"Today, 10:21 AM",Tesla bears capitulate for the near term,https://seekingalpha.com/news/3589120-tesla-be...,The bear camp on Tesla (TSLA +0.4%) is crackin...,error
2,"Yesterday, 1:16 PM",Tesla market cap crown brings focus on valuation,https://seekingalpha.com/news/3588818-tesla-ma...,Tesla shares got a lot of hype last week as th...,error
3,"Yesterday, 8:41 AM",Tesla 6 after JMP hikes price target to $1500,https://seekingalpha.com/news/3588673-teslaplu...,JMP Securities thinks sales volume at Tesla (N...,error
4,"Yesterday, 3:26 AM",Musk teases Tesla shortsellers with Short Shorts,https://seekingalpha.com/news/3588579-musk-tea...,Elon Musk has already offered to send a pair o...,error


In [ ]:
df3 = pd.read_csv(f'./data/{ticker}_articles.csv')
df3.head()

In [ ]:
df3['article'][0]

In [ ]:
w2v_model = Word2Vec(sentences=df3['article'],
                     min_count=1,
                 window=5,
                 size=100,
                 workers=4)

# w2v_model.build_vocab(sentences)
w2v_model.train(tweets, total_examples=w2v_model.corpus_count, epochs=w2v_model.iter)
w2v_model.init_sims(replace=True)

In [ ]:
driver = webdriver.Chrome('./chromedriver')
driver.get(url2)
driver.implicitly_wait(100)
article = driver.find_element_by_id("content-rail")
# assert "Python" in driver.title

# elem.clear()
# elem.send_keys("pycon")
# elem.send_keys(Keys.RETURN)
# assert "No results found." not in driver.page_source
driver.close()

In [ ]:
# The Article has comments, which for now I am going to take out. This code will separate the article into two sections with "Like this article" as the separator
sep = 'Recommended for you:'
article.text.split(sep,1)[1]

In [ ]:
r = urllib.request.Request(url, headers={'User-Agent': 'Chrome/83.0.4103.116'})

In [ ]:
html = urllib.request.urlopen(r)
time.sleep(10)

html = html.read()

In [ ]:

# Establishing the connection to the web page:
response = requests.get(url)
print(response.status_code)

# Pull the HTML string out of requests and convert it to a Python string.
html = response.text


In [ ]:
from requests_html import HTMLSession

session = HTMLSession()

r = session.get(url)

r.html.render()

In [ ]:
soup = BeautifulSoup(html,'lxml')

In [ ]:
import urllib3
import time
data = urllib3.urlopen(url)
time.sleep(10)
data = data.read()

In [ ]:
results_list = []

articles = soup.find_all('article', {'class':'title'})

for element in articles:
    # start a dictionary to store this item's data
    result = {}
    
    # get the title and full link/url
    a_href = element.find('a')
    if a_href:
        result['title'] = a_href.text   # element text
        result['link'] = a_href['href'] # href link
        
    # get the url domain
    span = element.find('span', {'class':'comhead'})
    if span:
        result['url'] = span.text.strip()[1:-1]
        
    # only store "full" rows of data
    if len(result) == 3:
        results_list.append(result)
        
results_list[0]